In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [5]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc

In [7]:
def model_train(X_train, y_train, X_test, y_test, num_iter, params_lgb):

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_test, y_test, reference=dtrain)
    model_lgb = lgb.train(params_lgb, dtrain, num_iter,
                          valid_sets=(dtrain, dvalid),
                          valid_names=('train', 'valid'),
                          early_stopping_rounds=150,
                          verbose_eval=20)

    return model_lgb

def CV(X, y, params_lgb, eval_method, shuffle=True, NFolds=5, num_iter=50):

    models = []
    y_folds = np.zeros((X.shape[0], 1))
    y_preds = np.zeros((X.shape[0], 1))

    I=0

    if eval_method=="kf":
        skf = KFold(n_splits=NFolds, random_state=42, shuffle=shuffle)
        for train_index, test_index in tqdm(skf.split(X, y)):
            print('[Fold %d/%d]' % (I + 1, NFolds))
            print('=' * 60)
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            model_lgb = model_train(X_train, y_train, X_test, y_test, num_iter, params_lgb)

            preds = model_lgb.predict(X_test)

            y_folds[test_index, 0] = y_test
            y_preds[test_index, 0] = preds
            models.append(model_lgb)

            I += 1

    return models, y_preds

In [12]:
X=train_df[main_attrs]
y=train_df[target[0]]

params_lgb ={
        'boosting_type': 'gbdt',
        'learning_rate': 0.015,
        'subsample': 1,
        'colsample_bytree': 0.2,
        'reg_alpha': 3,
        'reg_lambda': 1,
        'verbose': 1,
        'max_depth': -1,
        'seed':100,
        }

In [15]:
dtrain = lgb.Dataset(X, y)
model_lgb = lgb.train(params_lgb, dtrain, 5000, valid_sets=(dtrain),valid_names=('train'),
                          verbose_eval=20)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.089160 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13885
[LightGBM] [Info] Number of data points in the train set: 375734, number of used features: 59
[LightGBM] [Info] Start training from score 11.858568
[20]	train's l2: 59.0715
[40]	train's l2: 37.6757
[60]	train's l2: 24.6866
[80]	train's l2: 16.4785
[100]	train's l2: 11.6105
[120]	train's l2: 8.57081
[140]	train's l2: 6.52118
[160]	train's l2: 5.21089
[180]	train's l2: 4.35404
[200]	train's l2: 3.74528
[220]	train's l2: 3.30408
[240]	train's l2: 2.99226
[260]	train's l2: 2.74125
[280]	train's l2: 2.5398
[300]	train's l2: 2.38802
[320]	train's l2: 2.24832
[340]	train's l2: 2.14061
[360]	train's l2: 2.04669
[380]	train's l2: 1.96495
[400]	train's l2: 1.8803
[420]	train's l2: 1.81708
[440]	train's l2: 1.74741
[460]	train's l2: 1.69071
[480]	train's l2: 1.63433
[500]	train's l2: 1.59065
[520]	tr

In [16]:
y_pred = model_lgb.predict(test_df[main_attrs].values)

In [18]:
models, y_preds = CV(X, y, params_lgb, eval_method='kf', NFolds=10, num_iter=20000)

0it [00:00, ?it/s]

[Fold 1/10]
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13880
[LightGBM] [Info] Number of data points in the train set: 338160, number of used features: 59
[LightGBM] [Info] Start training from score 11.861901
Training until validation scores don't improve for 150 rounds
[20]	train's l2: 59.0683	valid's l2: 59.1549
[40]	train's l2: 37.6777	valid's l2: 37.7327
[60]	train's l2: 24.685	valid's l2: 24.7186
[80]	train's l2: 16.4706	valid's l2: 16.4904
[100]	train's l2: 11.6122	valid's l2: 11.6185
[120]	train's l2: 8.57211	valid's l2: 8.5735
[140]	train's l2: 6.52378	valid's l2: 6.52156
[160]	train's l2: 5.21367	valid's l2: 5.20889
[180]	train's l2: 4.35622	valid's l2: 4.35008
[200]	train's l2: 3.7453	valid's l2: 3.74056
[220]	train's l2: 3.30262	valid's l2: 3.2987
[240]	train's l2: 2.99091	valid's l2: 2.98683
[260]	train's l2: 2.73925	valid's 

[3240]	train's l2: 0.373189	valid's l2: 0.399115
[3260]	train's l2: 0.371483	valid's l2: 0.397447
[3280]	train's l2: 0.369399	valid's l2: 0.395439
[3300]	train's l2: 0.367748	valid's l2: 0.393863
[3320]	train's l2: 0.365816	valid's l2: 0.391961
[3340]	train's l2: 0.364038	valid's l2: 0.390238
[3360]	train's l2: 0.362056	valid's l2: 0.38831
[3380]	train's l2: 0.360193	valid's l2: 0.386488
[3400]	train's l2: 0.358416	valid's l2: 0.384714
[3420]	train's l2: 0.356557	valid's l2: 0.382862
[3440]	train's l2: 0.354924	valid's l2: 0.381268
[3460]	train's l2: 0.353095	valid's l2: 0.37948
[3480]	train's l2: 0.351315	valid's l2: 0.377754
[3500]	train's l2: 0.349572	valid's l2: 0.376033
[3520]	train's l2: 0.347893	valid's l2: 0.37436
[3540]	train's l2: 0.346384	valid's l2: 0.3729
[3560]	train's l2: 0.344503	valid's l2: 0.371053
[3580]	train's l2: 0.343002	valid's l2: 0.369526
[3600]	train's l2: 0.341415	valid's l2: 0.368007
[3620]	train's l2: 0.339746	valid's l2: 0.366363
[3640]	train's l2: 0.3381

[6620]	train's l2: 0.198262	valid's l2: 0.226576
[6640]	train's l2: 0.197738	valid's l2: 0.226056
[6660]	train's l2: 0.197213	valid's l2: 0.225524
[6680]	train's l2: 0.196659	valid's l2: 0.224971
[6700]	train's l2: 0.196166	valid's l2: 0.224486
[6720]	train's l2: 0.195624	valid's l2: 0.223927
[6740]	train's l2: 0.194984	valid's l2: 0.22325
[6760]	train's l2: 0.194507	valid's l2: 0.222773
[6780]	train's l2: 0.19401	valid's l2: 0.222277
[6800]	train's l2: 0.193438	valid's l2: 0.22169
[6820]	train's l2: 0.192909	valid's l2: 0.22116
[6840]	train's l2: 0.192384	valid's l2: 0.22063
[6860]	train's l2: 0.191846	valid's l2: 0.220068
[6880]	train's l2: 0.191272	valid's l2: 0.219479
[6900]	train's l2: 0.190688	valid's l2: 0.218869
[6920]	train's l2: 0.190183	valid's l2: 0.218363
[6940]	train's l2: 0.189613	valid's l2: 0.217789
[6960]	train's l2: 0.189133	valid's l2: 0.217321
[6980]	train's l2: 0.188599	valid's l2: 0.216807
[7000]	train's l2: 0.188143	valid's l2: 0.216341
[7020]	train's l2: 0.1876

[9980]	train's l2: 0.131483	valid's l2: 0.158874
[10000]	train's l2: 0.131229	valid's l2: 0.15862
[10020]	train's l2: 0.130975	valid's l2: 0.158367
[10040]	train's l2: 0.130719	valid's l2: 0.158114
[10060]	train's l2: 0.130455	valid's l2: 0.15783
[10080]	train's l2: 0.130184	valid's l2: 0.15756
[10100]	train's l2: 0.129934	valid's l2: 0.157304
[10120]	train's l2: 0.129628	valid's l2: 0.156986
[10140]	train's l2: 0.129366	valid's l2: 0.156717
[10160]	train's l2: 0.12908	valid's l2: 0.156424
[10180]	train's l2: 0.128829	valid's l2: 0.156167
[10200]	train's l2: 0.128574	valid's l2: 0.155915
[10220]	train's l2: 0.12831	valid's l2: 0.155647
[10240]	train's l2: 0.128078	valid's l2: 0.155411
[10260]	train's l2: 0.127826	valid's l2: 0.15515
[10280]	train's l2: 0.127599	valid's l2: 0.154915
[10300]	train's l2: 0.127353	valid's l2: 0.154662
[10320]	train's l2: 0.127071	valid's l2: 0.154363
[10340]	train's l2: 0.126833	valid's l2: 0.154131
[10360]	train's l2: 0.126531	valid's l2: 0.153809
[10380]

...

[10000]	train's l2: 0.131787	valid's l2: 0.156838
[10020]	train's l2: 0.131495	valid's l2: 0.156544
[10040]	train's l2: 0.13121	valid's l2: 0.156256
[10060]	train's l2: 0.130972	valid's l2: 0.156013
[10080]	train's l2: 0.130713	valid's l2: 0.155767
[10100]	train's l2: 0.130477	valid's l2: 0.155545
[10120]	train's l2: 0.130222	valid's l2: 0.155286
[10140]	train's l2: 0.129974	valid's l2: 0.15504
[10160]	train's l2: 0.129715	valid's l2: 0.154782
[10180]	train's l2: 0.129444	valid's l2: 0.154509
[10200]	train's l2: 0.129184	valid's l2: 0.154236
[10220]	train's l2: 0.128925	valid's l2: 0.153975
[10240]	train's l2: 0.128693	valid's l2: 0.153744
[10260]	train's l2: 0.128424	valid's l2: 0.153493
[10280]	train's l2: 0.128184	valid's l2: 0.153252
[10300]	train's l2: 0.127964	valid's l2: 0.153033
[10320]	train's l2: 0.127713	valid's l2: 0.152781
[10340]	train's l2: 0.127403	valid's l2: 0.15244
[10360]	train's l2: 0.127106	valid's l2: 0.152122
[10380]	train's l2: 0.126881	valid's l2: 0.15189
[104

[13320]	train's l2: 0.097096	valid's l2: 0.121582
[13340]	train's l2: 0.096911	valid's l2: 0.121392
[13360]	train's l2: 0.0967408	valid's l2: 0.121213
[13380]	train's l2: 0.0965479	valid's l2: 0.121008
[13400]	train's l2: 0.0963853	valid's l2: 0.120834
[13420]	train's l2: 0.0962393	valid's l2: 0.120685
[13440]	train's l2: 0.0960856	valid's l2: 0.120517
[13460]	train's l2: 0.0959253	valid's l2: 0.120357
[13480]	train's l2: 0.0957661	valid's l2: 0.120194
[13500]	train's l2: 0.0956019	valid's l2: 0.120024
[13520]	train's l2: 0.0954647	valid's l2: 0.119887
[13540]	train's l2: 0.0952818	valid's l2: 0.119689
[13560]	train's l2: 0.0951289	valid's l2: 0.119533
[13580]	train's l2: 0.0949855	valid's l2: 0.119385
[13600]	train's l2: 0.0948011	valid's l2: 0.119188
[13620]	train's l2: 0.0946578	valid's l2: 0.119042
[13640]	train's l2: 0.0945216	valid's l2: 0.118897
[13660]	train's l2: 0.094351	valid's l2: 0.118715
[13680]	train's l2: 0.094192	valid's l2: 0.118547
[13700]	train's l2: 0.0940458	valid

[16560]	train's l2: 0.0762159	valid's l2: 0.099955
[16580]	train's l2: 0.0761156	valid's l2: 0.0998526
[16600]	train's l2: 0.076	valid's l2: 0.0997336
[16620]	train's l2: 0.0758982	valid's l2: 0.09963
[16640]	train's l2: 0.0757952	valid's l2: 0.0995275
[16660]	train's l2: 0.0757019	valid's l2: 0.0994376
[16680]	train's l2: 0.0756124	valid's l2: 0.0993476
[16700]	train's l2: 0.0754917	valid's l2: 0.0992217
[16720]	train's l2: 0.0753838	valid's l2: 0.0991085
[16740]	train's l2: 0.0752853	valid's l2: 0.0990034
[16760]	train's l2: 0.0751881	valid's l2: 0.0989015
[16780]	train's l2: 0.0750901	valid's l2: 0.0988021
[16800]	train's l2: 0.0749943	valid's l2: 0.0987067
[16820]	train's l2: 0.0749003	valid's l2: 0.098614
[16840]	train's l2: 0.074798	valid's l2: 0.0985071
[16860]	train's l2: 0.0746984	valid's l2: 0.0984032
[16880]	train's l2: 0.0746065	valid's l2: 0.0983157
[16900]	train's l2: 0.0745111	valid's l2: 0.0982177
[16920]	train's l2: 0.0744111	valid's l2: 0.0981144
[16940]	train's l2: 0

[19740]	train's l2: 0.0624671	valid's l2: 0.0856077
[19760]	train's l2: 0.0624008	valid's l2: 0.0855398
[19780]	train's l2: 0.0623342	valid's l2: 0.0854656
[19800]	train's l2: 0.0622582	valid's l2: 0.0853868
[19820]	train's l2: 0.0621598	valid's l2: 0.08528
[19840]	train's l2: 0.0620839	valid's l2: 0.0851985
[19860]	train's l2: 0.0620115	valid's l2: 0.0851244
[19880]	train's l2: 0.0619303	valid's l2: 0.085034
[19900]	train's l2: 0.061864	valid's l2: 0.0849635
[19920]	train's l2: 0.06179	valid's l2: 0.0848816
[19940]	train's l2: 0.0617216	valid's l2: 0.0848075
[19960]	train's l2: 0.0616407	valid's l2: 0.0847202
[19980]	train's l2: 0.0615783	valid's l2: 0.0846541
[20000]	train's l2: 0.0615026	valid's l2: 0.0845751
Did not meet early stopping. Best iteration is:
[20000]	train's l2: 0.0615026	valid's l2: 0.0845751


10it [2:51:24, 1028.50s/it]


In [21]:
dot(y_preds, s)/(norm(y_best)*norm(s))

0         28.744480
1         28.370585
2         28.133059
3         28.256798
4         28.372353
            ...    
375729    17.150954
375730    16.962051
375731    16.915474
375732    16.536761
375733    15.910995
Name: contest-tmp2m-14d__tmp2m, Length: 375734, dtype: float64

In [24]:
pred_train = y_preds[:, 0]
train_y = np.array(train_df[target[0]].tolist())
cos_sim = dot(train_y, pred_train)/(norm(pred_train)*norm(train_y))
print("cos_sim with best submission:", cos_sim)

cos_sim with best submission: 0.9998223707043576


In [26]:
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

In [19]:
result_df = pd.read_csv('./sample_solution.csv')
s = 0
for model in models:
    test_pred = model.predict(test_df[main_attrs].values, num_iteration=model.best_iteration)
    s += (test_pred / len(models))

cos_sim with best submission: 0.9981139848955439


In [31]:
t = s * 0.1 + y_best * 0.9
result_df['contest-tmp2m-14d__tmp2m'] = t
result_df.to_csv('submission_light_gbm_v1.csv', index=False)
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, t)/(norm(y_best)*norm(t))
print("cos_sim with best submission:", cos_sim)

cos_sim with best submission: 0.9999807905520286
